# Pruebas Trabajo con MLPs

In [1]:
from keras.optimizers import SGD
from tensorflow import keras
import cv2
import numpy as np
import os
from sklearn.cluster import MiniBatchKMeans
from keras import Sequential
from sklearn.base import BaseEstimator, TransformerMixin
from keras.layers import Dense, Input, Dropout, Flatten
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve
import pandas as pd

2024-12-05 13:40:34.493407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733402434.511731   43208 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733402434.517124   43208 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 13:40:34.537222: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def show_results_gs(gs, sort=False):
    # Extraer los resultados del GridSearchCV
    results = gs.cv_results_

    # Crear un DataFrame con los parámetros y los resultados medios
    df_results = pd.DataFrame({
        'mean_test_score': results['mean_test_score'],
        'params': results['params']
    })

    # Ordenar los resultados por el resultado promedio (opcional)
    if sort:
        df_results = df_results.sort_values(by='mean_test_score', ascending=False)

    # Mostrar los resultados
    print(df_results)

In [3]:
dataset_path = '/home/pablo/Desktop/tercero/mdp/trabajo/HANDS'   # Cambiar por la carpeta en la que se encuentran las imágenes
 
jpgFiles = [fJPG for fJPG in os.listdir(dataset_path) if fJPG.endswith('.jpg')]

bufferImages = []
y = []

# Extraer 50 patches aleatorios de cada imagen
for filename in jpgFiles:
    img_path = os.path.join(dataset_path, filename)
    label = filename.split('.')[0][-2:]
    y.append(label)
    with open(img_path) as infile:
        # Obtener etiqueta
        img_label = filename[5:7] 
        # Leer imagen
        img = cv2.imread(img_path)
        # Convertir imagen de BGR a RGB
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        bufferImages.append(img_gray)
print(len(bufferImages))
print('Comprobación:', all([x in ['RB', 'RF', 'LB', 'LF'] for x in y]))

200
Comprobación: True


In [4]:
class CustomPreprocessorSamePatches(BaseEstimator, TransformerMixin):
    def __init__(self, patchSize=10, nClusters = 81, max_patches=50):
        self.patchSize = patchSize
        self.nClusters = nClusters
        self.max_patches = max_patches

    def fit(self, X, y=None):
        return self       
    
    def extract_centric_patches(self, img, patch_size, num_patches, seed=42):
        assert img.shape[0] == 150 and img.shape[1] == 150, "La imagen debe tener resolución 150x150."
        assert patch_size > 0, "El tamaño del patch debe ser mayor que 0."
        assert patch_size * num_patches <= 150 * 150, "Demasiados patches para esta resolución."
        
        # Calcular la cuadrícula de celdas no superpuestas
        grid_rows = 150 // patch_size
        grid_cols = 150 // patch_size
        total_cells = grid_rows * grid_cols

        assert num_patches <= total_cells, "Demasiados patches para el tamaño del patch y la resolución de la imagen."
        
        # Generar las posiciones de la cuadrícula
        all_positions = [(i * patch_size, j * patch_size) for i in range(grid_rows) for j in range(grid_cols)]

        # Calcular las distancias de las celdas al centro de la imagen
        center = (150 // 2, 150 // 2)
        distances = [
            np.sqrt((row + patch_size // 2 - center[0])**2 + (col + patch_size // 2 - center[1])**2)
            for row, col in all_positions
        ]

        # Ordenar las posiciones por cercanía al centro
        sorted_positions = [pos for _, pos in sorted(zip(distances, all_positions))]
        
        # Seleccionar los patches más céntricos
        np.random.seed(seed)
        selected_positions = sorted_positions[:num_patches]
        
        # Extraer los patches en las posiciones seleccionadas
        patches = [img[row:row + patch_size, col:col + patch_size] for row, col in selected_positions]

        #self.visualize_disjoint_patches(img, patches, selected_positions, patch_size)
        
        return np.array(patches)


    def bovw(self, images, patchSize, nClusters, maxPatches):
        bufferData = []
        for i in images:
            resized_img = cv2.resize(i, (150, 150), interpolation=cv2.INTER_AREA)
            patches = self.extract_centric_patches(resized_img, patchSize, num_patches=maxPatches, seed=27)
            patches = np.reshape(patches, (len(patches), -1))
            bufferData.append(patches)
            
        bufferData = np.array(bufferData)
        print(bufferData.shape)
        return bufferData

    def transform(self, images):
        # Aplica tu función de preprocesamiento aquí
        # Por ejemplo:
        # return your_custom_function(X, param1=self.param1)
        return self.bovw(images, self.patchSize, self.nClusters, self.max_patches)  # Devuelve los datos preprocesados

In [5]:
model = Sequential()
model.add(Input(shape=(50, 300)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer = SGD(), loss='categorical_crossentropy',
                metrics=['accuracy'])

I0000 00:00:1733402437.533376   43208 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5787 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [6]:
cp = CustomPreprocessorSamePatches()
resul = cp.transform(bufferImages)

(200, 50, 300)


In [7]:
enc = LabelEncoder()
y_cat = enc.fit_transform(y)
y_one_hot = to_categorical(y_cat, 4)
x_train, x_test, y_train, y_test = train_test_split(resul, y_one_hot, test_size=0.2)

history = model.fit(x_train, y_train, epochs=60, batch_size=16)

Epoch 1/60


I0000 00:00:1733402438.835599   43524 service.cc:148] XLA service 0x78e14c003580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733402438.835625   43524 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-12-05 13:40:38.852517: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733402438.898968   43524 cuda_dnn.cc:529] Loaded cuDNN version 90300


10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2546 - loss: nan        
Epoch 2/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2400 - loss: nan
Epoch 3/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3008 - loss: nan
Epoch 4/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2579 - loss: nan
Epoch 5/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2302 - loss: nan
Epoch 6/60
 2/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2344 - loss: nan 

I0000 00:00:1733402441.061391   43524 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2674 - loss: nan
Epoch 7/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2676 - loss: nan
Epoch 8/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2464 - loss: nan
Epoch 9/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2268 - loss: nan
Epoch 10/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2487 - loss: nan
Epoch 11/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2848 - loss: nan
Epoch 12/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2983 - loss: nan
Epoch 13/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2742 - loss: nan
Epoch 14/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2445 - loss: nan
Epoch 15/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2874 - loss: nan
Epoch 16/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2436 - loss: nan
Epoch 17/60
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2280 - loss: nan
Epoch 18/60
10/10 ━━━━━━━━━━━━━

Desastre

## Pruebas con BoVW

In [8]:
class CustomBoVWPreprocessorSamePatches(BaseEstimator, TransformerMixin):
    def __init__(self, patchSize=10, nClusters = 81, max_patches=50):
        self.patchSize = patchSize
        self.nClusters = nClusters
        self.max_patches = max_patches

    def fit(self, X, y=None):
        return self     
    
    def extract_centric_patches(self, img, patch_size, num_patches, seed=42):
        assert img.shape[0] == 150 and img.shape[1] == 150, "La imagen debe tener resolución 150x150."
        assert patch_size > 0, "El tamaño del patch debe ser mayor que 0."
        assert patch_size * num_patches <= 150 * 150, "Demasiados patches para esta resolución."
        
        # Calcular la cuadrícula de celdas no superpuestas
        grid_rows = 150 // patch_size
        grid_cols = 150 // patch_size
        total_cells = grid_rows * grid_cols

        assert num_patches <= total_cells, "Demasiados patches para el tamaño del patch y la resolución de la imagen."
        
        # Generar las posiciones de la cuadrícula
        all_positions = [(i * patch_size, j * patch_size) for i in range(grid_rows) for j in range(grid_cols)]

        # Calcular las distancias de las celdas al centro de la imagen
        center = (150 // 2, 150 // 2)
        distances = [
            np.sqrt((row + patch_size // 2 - center[0])**2 + (col + patch_size // 2 - center[1])**2)
            for row, col in all_positions
        ]

        # Ordenar las posiciones por cercanía al centro
        sorted_positions = [pos for _, pos in sorted(zip(distances, all_positions))]
        
        # Seleccionar los patches más céntricos
        np.random.seed(seed)
        selected_positions = sorted_positions[:num_patches]
        
        # Extraer los patches en las posiciones seleccionadas
        patches = [img[row:row + patch_size, col:col + patch_size] for row, col in selected_positions]

        #self.visualize_disjoint_patches(img, patches, selected_positions, patch_size)
        
        return np.array(patches)


    def bovw(self, images, patchSize, nClusters, maxPatches):
        bufferData = []
        for i in images:
            resized_img = cv2.resize(i, (150, 150), interpolation=cv2.INTER_AREA)
            patches = self.extract_centric_patches(resized_img, patchSize, num_patches=maxPatches, seed=27) # shape = (maxPatches,5,5,3)
            patches = np.reshape(patches, (len(patches), -1)) # shape = (maxPatches,75)
            bufferData.append(patches)
        dataIm = np.concatenate(bufferData, axis=0) # shape = (200*maxPatches,75)  n patches x 200 imágenes
        dataIm = dataIm.astype(float)
        dataIm -= np.mean(dataIm, axis=0)
        dataIm /= np.std(dataIm, axis=0)
        kmeans = MiniBatchKMeans(n_clusters=nClusters, random_state=27, verbose=False)
        kmeans.partial_fit(dataIm)

        X = np.reshape(kmeans.labels_ , (len(images),maxPatches)) # Indice de cluster de cada patch. shape = (200,maxPatches)
        resul = [np.bincount(row, minlength=nClusters) for row in X]
        resul = np.array(resul)
        return resul

    def transform(self, images):
        # Aplica tu función de preprocesamiento aquí
        # Por ejemplo:
        # return your_custom_function(X, param1=self.param1)
        return self.bovw(images, self.patchSize, self.nClusters, self.max_patches)  # Devuelve los datos preprocesados


In [9]:
enc = LabelEncoder()
X = bufferImages
y_cod = enc.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(X, y_cod, test_size=0.2)

In [20]:
svm = SVC()
pca = PCA()
proc = CustomBoVWPreprocessorSamePatches()

pipe = Pipeline([('proc', proc), ('pca', pca), ('svm', svm)])
G = {'svm__C': [0.1, 1], 'svm__kernel':['rbf'], 'svm__gamma': [0.1, 1], 'pca__n_components': [10]}
gs = GridSearchCV(pipe, G, cv=4, refit=True, verbose=10)

In [21]:
gs.fit(x_train, y_train)

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV 1/4; 1/4] START pca__n_components=10, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 1/4; 1/4] END pca__n_components=10, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.8s
[CV 2/4; 1/4] START pca__n_components=10, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 2/4; 1/4] END pca__n_components=10, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.6s
[CV 3/4; 1/4] START pca__n_components=10, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 3/4; 1/4] END pca__n_components=10, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.6s
[CV 4/4; 1/4] START pca__n_components=10, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 4/4; 1/4] END pca__n_components=10, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.6s
[CV 1/4; 2/4] START pca__n_components=10, svm__C=0.1, svm__gamma=1, svm__kernel=rbf
[CV 1/4; 2/4] END pca__n_components=10, svm_

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('proc',
                                        CustomBoVWPreprocessorSamePatches()),
                                       ('pca', PCA()), ('svm', SVC())]),
             param_grid={'pca__n_components': [10], 'svm__C': [0.1, 1],
                         'svm__gamma': [0.1, 1], 'svm__kernel': ['rbf']},
             verbose=10)

In [22]:
print('Los resultados en train son:')
show_results_gs(gs)
print('Los mejores parámetros son:')
print(gs.best_params_)
print('Con score:')
print(gs.best_score_)
print('Los resultados en test:')
y_preds = gs.predict(x_test)
print(classification_report(y_test, y_preds))

Los resultados en train son:
   mean_test_score                                             params
0          0.25000  {'pca__n_components': 10, 'svm__C': 0.1, 'svm_...
1          0.25000  {'pca__n_components': 10, 'svm__C': 0.1, 'svm_...
2          0.31875  {'pca__n_components': 10, 'svm__C': 1, 'svm__g...
3          0.26875  {'pca__n_components': 10, 'svm__C': 1, 'svm__g...
Los mejores parámetros son:
{'pca__n_components': 10, 'svm__C': 1, 'svm__gamma': 0.1, 'svm__kernel': 'rbf'}
Con score:
0.31875
Los resultados en test:
              precision    recall  f1-score   support

           0       0.26      0.82      0.40        11
           1       1.00      0.11      0.20         9
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00        10

    accuracy                           0.25        40
   macro avg       0.32      0.23      0.15        40
weighted avg       0.30      0.25      0.15        40



Terriblemente mal